<a href="https://colab.research.google.com/github/MohammedQaraad/DeepleerningTF/blob/master/mnistmanualTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#هنا نقوم بقراء الداتا سيت اللي اسمها mnist 
#والموجودة في ملف اسمه data
#نقوم بقراءة الداتا سيت كاملة 
#طبعا اي داتا سيت تقسم الى ثلاثة اقسام  train  - validation  - test 
# هذه الداتا سيت مقسمة جاهزة 
#الداتا سيت هذه عبارة عن صور للكتابة بخط اليد للراقام من 0 - 9
#بمعنى ان لدينا  10 كلاس ليبل في ال output layer 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/")
#اصبخت الداتا سيت مخزنة في متغير mnist 

In [24]:
#لمعرفة عدد الصور المخصصة لكي يقوم النموذج بالتدرب عليها في الداتا سيت 
print('number of training data   {}'.format (mnist.train.num_examples))

#لمعرفة عدد الصور المخصصة لكي يقوم النموذج بعملية ضبط الباراميترات عليها في الداتا سيت 
print('number of validation data   {}'.format (mnist.validation.num_examples))

#لمعرفة عدد الصور المخصصة لفحص النموذج ما اذا كان يعمل بشكل سليم او لا  
print('number of testing  data   {}'.format (mnist.test.num_examples))

#حجم الداتا ست كاملة 
print('total dataset size  {} '.format(mnist.train.num_examples + mnist.test.num_examples + mnist.validation.num_examples))


print(mnist.train.labels.shape)
mnist.train.labels[0]


number of training data   55000
number of validation data   5000
number of testing  data   10000
total dataset size  70000 
(55000,)


9

In [0]:

n_epochs = 400
batch_size = 50
#هذه المتغيرات بتحدد حاجتين الاولى عدد المرات التي بيشتغل عليها النموذج على الداتا ست بتاع التدريب  وهي اربعمئة مرة 
#المتغير الاخر بما ان الداتا سيت الخاصة بالتدريب كبيرة فيتم اخذ اربعين صورة والتعلم عليها ومن ثم اربعين اخرى يعني ما ياخذ كل الداتا مرة واحدة 

In [17]:
import tensorflow as tf

from datetime import datetime
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir='./graphs'
logdir="{}/run-{}/".format(root_logdir, now)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())
logdir

'./graphs/run-20190610223807/'

In [18]:
# هنا نقوم ببناء الشبكة العصبية اي النموذج 
n_inputs = 28*28 # MNIST  input layer 
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

from tensorflow.contrib.layers import fully_connected

with tf.name_scope("dnn"):
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, scope="outputs",activation_fn=None)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()



Instructions for updating:
Colocations handled automatically by placer.


In [20]:
with tf.Session() as sess:
    init.run()
    for epoch in range(5): #
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
    print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./my_model_final.ckpt")

4 Train accuracy: 0.94 Test accuracy: 0.9434
